In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import re

In [2]:
# wczytanie i zlepienie w jeden dataset z dodatkową kolumną do rozróżnienia zbiorów
tt = pd.concat([
    pd.read_hdf('./train.car_price.h5').drop(columns='price_details').assign(train=True),
    pd.read_hdf('./test.car_price.h5').assign(train=False)
])


In [3]:
MT_LOOKUP = {
    'styczeń': 1, 'luty': 2, 'marzec': 3, 'kwiecień': 4, 'maj': 5, 'czerwiec': 6, 'lipiec': 7, 'sierpień': 8, 'wrzesień': 9, 'październik': 10, 'listopad': 11, 'grudzień': 12,
    'january': 1, 'february': 2, 'march': 3, 'april': 4, 'may': 5, 'june': 6, 'july': 7, 'august': 8, 'september': 9, 'october': 10, 'november': 11, 'december': 12
}

# funkcje pomocnicze
def long_date(s):
    if s is None:
        return None
    d = s.lower().split(' ')
    if (len(d) < 3):
        d = ['1'] + d
    
    return dt.date(int(d[2]), MT_LOOKUP[d[1]], int(d[0]))

def as_int(m):
    return -1 if m is None else int(m.replace(' ', '').replace('g/km', '').replace('km', '').replace('cm3', '').replace('HP', '').replace('KM', '').replace('PLN', ''))

def as_float(m):
    return -1 if m is None else float(m.replace(' ', '').replace('PLN', '').replace(',', '.'))

def as_bool(m):
    return False if m is None else True 

def as_obj(m):
    return '-1' if m is None else m 

def param(df, c1, c2, transform=lambda x: x, data_type=np.object):
    return df[c1].fillna(df[c2]).apply(transform).astype(data_type)


# wstępne przygotowanie parametrów, łącząc wartości z dwóch kolumn, jeśli występuje polska i angielska wersja kolumny
def prepare_dataset(df):
    df = df.copy()

    t = np.bool
    df['p_damaged'] = param(df, 'param_uszkodzony', 'param_damaged', as_bool, t)
    df['p_pearl'] = param(df, 'param_pearl', 'param_perłowy', as_bool, t)
    df['p_metallic'] = param(df, 'param_metalik', 'param_metallic', as_bool, t)
    df['p_original_owner'] = param(df, 'param_original-owner', 'param_pierwszy-właściciel', as_bool, t)
    df['p_registered_in_poland'] = param(df, 'param_registered-in-poland', 'param_zarejestrowany-w-polsce', as_bool, t)
    df['p_no_accident'] = param(df, 'param_bezwypadkowy', 'param_no-accident', as_bool, t)
    df['p_particle_filter'] = param(df, 'param_particle-filter', 'param_filtr-cząstek-stałych', as_bool, t)
    df['p_leasing'] = param(df, 'param_leasing', 'param_leasing-concession', as_bool, t)
    df['p_acrylic'] = param(df, 'param_acrylic', 'param_akryl-(niemetalizowany)', as_bool, t)
    df['p_financing_option'] = param(df, 'param_financing-option', 'param_możliwość-finansowania', as_bool, t)
    df['p_service-record'] = param(df, 'param_service-record', 'param_serwisowany-w-aso', as_bool, t)
    df['p_vat_free'] = param(df, 'param_vat-free', 'param_vat-marża', as_bool, t)
    df['p_matt'] = param(df, 'param_matowy', 'param_matowy', as_bool, t)
    df['p_tuning'] = param(df, 'param_tuning', 'param_tuning', as_bool, t)
    df['p_engilish_version'] = param(df, 'param_kierownica-po-prawej-(anglik)', 'param_kierownica-po-prawej-(anglik)', as_bool, t)
    df['p_truck'] = param(df, 'param_homologacja-ciężarowa', 'param_homologacja-ciężarowa', as_bool, t)
    df['p_vintage'] = param(df, 'param_zarejestrowany-jako-zabytek', 'param_zarejestrowany-jako-zabytek', as_bool, t)
    df['p_vat_invoice'] = param(df, 'param_faktura-vat', 'param_faktura-vat', as_bool, t)
    df['p_vat_discount'] = param(df, 'param_vat-discount', 'param_vat-discount', as_bool, t)

    t = np.int
    df['p_seats'] = param(df, 'param_liczba-miejsc', 'param_nr-of-seats', as_int, t)
    df['p_doors'] = param(df, 'param_liczba-drzwi', 'param_door-count', as_int, t)
    df['p_year'] = param(df, 'param_year', 'param_rok-produkcji', as_int, t)
    df['p_mileage'] = param(df, 'param_mileage', 'param_przebieg', as_int, t)
    df['p_co2_emission'] = param(df, 'param_emisja-co2', 'param_co2-emissions', as_int, t)
    df['p_engine_capacity'] = param(df, 'param_engine-capacity', 'param_pojemność-skokowa', as_int, t)
    df['p_engine_power'] = param(df, 'param_engine-power', 'param_moc', as_int, t)

    t = np.float
    df['p_monthly_payment'] = param(df, 'param_miesięczna-rata', 'param_monthly-payment-value', as_float, t)
    df['p_rates_left'] = param(df, 'param_liczba-pozostałych-rat', 'param_liczba-pozostałych-rat', as_float, t)
    df['p_redemption_value'] = param(df, 'param_wartość-wykupu', 'param_wartość-wykupu', as_float, t)
    df['p_initial_payment'] = param(df, 'param_opłata-początkowa', 'param_opłata-początkowa', as_float, t)

    t = np.object
    df['p_model'] = param(df, 'param_model-pojazdu', 'param_model', as_obj, t)
    df['p_make'] = param(df, 'param_marka-pojazdu', 'param_make', as_obj, t)
    df['p_transmission'] = param(df, 'param_transmission', 'param_napęd', as_obj, t)
    df['p_fuel_type'] = param(df, 'param_rodzaj-paliwa', 'param_fuel-type', as_obj, t)
    df['p_body_type'] = param(df, 'param_body-type', 'param_typ', as_obj, t)
    df['p_version'] = param(df, 'param_version', 'param_wersja', as_obj, t)
    df['p_offered_by'] = param(df, 'param_oferta-od', 'param_oferta-od', as_obj, t)
    df['p_vin'] = param(df, 'param_vin', 'param_vin', as_obj, t)
    df['p_engine_code'] = param(df, 'param_engine-code', 'param_kod-silnika', as_obj, t)
    df['p_origin_country'] = param(df, 'param_country-of-origin', 'param_kraj-pochodzenia', as_obj, t)
    df['p_gearbox'] = param(df, 'param_gearbox', 'param_skrzynia-biegów', as_obj, t)
    df['p_color'] = param(df, 'param_kolor', 'param_color', as_obj, t)
    df['p_state'] = param(df, 'param_stan', 'param_stan', as_obj, t)
    df['p_category'] = param(df, 'param_kategoria', 'param_kategoria', as_obj, t)
    
    df['p_first_registration'] = pd.to_datetime(param(df, 'param_first-registration', 'param_pierwsza-rejestracja', long_date, t))
    
    df['seller_address'] = df['seller_address'].fillna('-1')
    df['seller_name'] = df['seller_name'].fillna('-1')
    df['seller_type'] = df['seller_type'].fillna('-1')

    return df[[c for c in df.columns if not c.startswith('param_')]]

tt = prepare_dataset(tt)

In [4]:
# pomocnicze zapisywanie i odczytywanie gdybym coś później namieszał ;) 

# tt.to_hdf('car_price.dataset.v1.h5', 'data')
# tt = pd.read_hdf('car_price.dataset.v1.h5')


/Users/pawelg/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2505: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['breadcrumb', 'created_at', 'price_currency', 'seller_address',
       'seller_name', 'seller_type', 'p_model', 'p_make', 'p_transmission',
       'p_fuel_type', 'p_body_type', 'p_version', 'p_offered_by', 'p_vin',
       'p_engine_code', 'p_origin_country', 'p_gearbox', 'p_color', 'p_state',
       'p_category'],
      dtype='object')]

  encoding=encoding,


In [5]:
# uzupełnie brakujących danych dla: marki, modelu i kategorii

def resolve_breadcrumb(df):
    df = df.copy()  
    def _make(r):
        return r[1]

    def _cat(r):
        if r[0] == 'Cars':
            return 'Osobowe'
        return r[0]
    
    def _model(r):
        mk = r[1]
        return r[2].replace(f'{mk}-', '')

    df['p_make'] = np.where(df['p_make'] == '-1', df['breadcrumb'].apply(_make), df['p_make'])
    df['p_model'] = np.where(df['p_model'] == '-1', df['breadcrumb'].apply(_model), df['p_model'])
    df['p_category'] = np.where(df['p_category'] == '-1', df['breadcrumb'].apply(_cat), df['p_category'])
    
    return df

tt = resolve_breadcrumb(tt)

In [6]:
# liczba dni od kiedy wisi ogłoszenie

def resolve_creation_dt(df):
    df = df.copy()  
    MN = { 'stycznia': 1, 'lutego': 2, 'marca': 3, 'kwietnia': 4, 'maja': 5, 'czerwca': 6, 'lipca': 7, 'sierpnia': 8, 'września': 9, 'października': 10, 'listopada': 11, 'grudnia': 12}

    def _created_at(r):
        if r is None:
            return None
        d = r.replace(',','').split(' ')
        return dt.date(year=int(d[3]), month=MN[d[2]], day=int(d[1])) 
    
    df['posted_dt'] = pd.to_datetime(df['created_at'].apply(_created_at))
    df['cf_posted_days'] = ((df['posted_dt'].max() - df['posted_dt']).dt.days + 1).fillna(-1)
    return df

tt = resolve_creation_dt(tt)

In [7]:
# tt.to_hdf('car_price.dataset.v2.h5', 'data')
# tt = pd.read_hdf('car_price.dataset.v2.h5')


/Users/pawelg/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2505: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block4_values] [items->Index(['breadcrumb', 'created_at', 'price_currency', 'seller_address',
       'seller_name', 'seller_type', 'p_model', 'p_make', 'p_transmission',
       'p_fuel_type', 'p_body_type', 'p_version', 'p_offered_by', 'p_vin',
       'p_engine_code', 'p_origin_country', 'p_gearbox', 'p_color', 'p_state',
       'p_category'],
      dtype='object')]

  encoding=encoding,


In [8]:
# wyciągnięcie przydatnych danych z wersji (początek i koniec danego modelu i czy podany rocznik mieści się w tym przedziale)

def decode_version(df):
    df = df.copy()
    
    p = re.compile('(^.*)\s+\((\d+)\s*-\s*(.*)\)')

    def _resolve_ver(s):
        if s == 'E8 i starsze (-1987)':
            return ['E8', 1900, 1987]
        
        m = p.match(s)
        if m:
            ver, min_year, max_year = m.group(1), m.group(2), m.group(3)
            
            if max_year == '':
                max_year = '2019'
                
            return [ver, int(min_year), int(max_year)]
        return [s, -1, -1]
    
    k = df['p_version'].apply(_resolve_ver)
    df['cf_version_name'] = k.apply(lambda x: x[0])
    df['cf_version_min_year'] = k.apply(lambda x: x[1])
    df['cf_version_max_year'] = k.apply(lambda x: x[2])
    df['cf_year_in_version_range'] = (df['p_year'] != -1) & (df['cf_version_min_year'] != -1) & (df['p_year'] >= df['cf_version_min_year']) & (df['p_year'] <= df['cf_version_max_year'])
    return df

tt = decode_version(tt)


In [9]:
# policzenie liczby dni pierwszej rejestracji w stosunku do rocznika

def resolve_registration_days(df):
    df = df.copy()
    # ręczne uzupełnienie paru ewidentnie błędnych danych
    df.loc[37093, 'p_year'] = 1993
    df.loc[47682, 'p_year'] = 1998
    df.loc[123600, 'p_year'] = 2003
    df.loc[95514, 'p_year'] = 2004
    df.loc[10699, 'p_year'] = 1993
    df.loc[150795, 'p_year'] = 2004

    df.loc[68031, 'p_first_registration'] = '2017-01-01'
    df.loc[103136, 'p_first_registration'] = '2014-01-19'
    df.loc[131238, 'p_first_registration'] = '2017-06-01'
    df.loc[53998, 'p_first_registration'] = '2006-07-31'
    
    def year_to_date(d):
        return None if d is -1 else pd.Timestamp(dt.date(year=d, month=1, day=1))
        
    df['p_first_registration'] = pd.to_datetime(df['p_first_registration'])    
        
    ref = df['p_year'].apply(year_to_date)
    df['p_first_registration'] = np.where(df['p_first_registration'] > df['posted_dt'], df['posted_dt'], df['p_first_registration'])
    
    df['cf_year_vs_reg_in_days'] = ((df['p_first_registration'] - df['p_year'].apply(year_to_date)).dt.days).fillna(-10000).astype(np.int16)
    return df

tt = resolve_registration_days(tt)

In [10]:
# wyciągnięcie paru informacji z nazwy sprzedawcy i adresu

def resolve_car_seller(df):
    df = df.copy()
    df['cf_used_car_delear'] = df['seller_name'].apply(lambda x: 'komis' in x.lower())                                      # komis
    df['cf_used_car_seller'] = df['seller_name'].apply(lambda x: 'używ' in x.lower() or 'uzyw' in x.lower())                # używki
    df['cf_damaged_car_seller'] = df['seller_name'].apply(lambda x: 'powypad' in x.lower())                                 # powypadkowe
    df['cf_authorized_car_seller'] = df['seller_name'].apply(lambda x: 'autoryzow' in x.lower())                            # autoryzowany sprzedawca
    df['cf_imported_car_seller'] = df['seller_name'].apply(lambda x: 'import' in x.lower() or 'sprowadz' in x.lower())      # importer
    df['cf_leased_car_seller'] = df['seller_name'].apply(lambda x: 'poleas' in x.lower())                                   # poleasingowe
    
    df['cf_seller_address_krakow'] = df['seller_address'].apply(lambda x: 'krakow' in x.lower() or 'kraków' in x.lower())   # kraków
    df['cf_seller_address_warszawa'] = df['seller_address'].apply(lambda x: 'warszawa' in x.lower())                        # wawa
    df['cf_seller_address_gdansk'] = df['seller_address'].apply(lambda x: 'gdańsk' in x.lower() or 'gdansk' in x.lower())   # gdańsk
    df['cf_seller_address_poznan'] = df['seller_address'].apply(lambda x: 'poznań' in x.lower() or 'poznan' in x.lower())   # poznań
    df['cf_seller_address_wroclaw'] = df['seller_address'].apply(lambda x: 'wrocław' in x.lower() or 'wroclaw' in x.lower()) # wrocek
    return df

tt = resolve_car_seller(tt)


In [11]:
# skoro już wiemy coś o sprzedawcy, to można spróbować usupełnić kraj pochodzenia

def adj_orig_country(df):
    df = df.copy()
    df.loc[df['seller_name'].str.lower().str.contains('niemiec') & (df['p_origin_country'] == '-1'), 'p_origin_country'] = 'Niemcy'
    df.loc[df['cf_authorized_car_seller'] & ~df['cf_imported_car_seller'] & ~df['cf_used_car_seller'] & ~df['cf_damaged_car_seller'] & (df['p_origin_country'] == '-1'), 'p_origin_country'] = 'Polska'
    return df

tt = adj_orig_country(tt)

In [12]:
tt.to_hdf('car_price.dataset.v3.h5', 'data')
tt = pd.read_hdf('car_price.dataset.v3.h5')


/Users/pawelg/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2505: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block5_values] [items->Index(['breadcrumb', 'created_at', 'price_currency', 'seller_address',
       'seller_name', 'seller_type', 'p_model', 'p_make', 'p_transmission',
       'p_fuel_type', 'p_body_type', 'p_version', 'p_offered_by', 'p_vin',
       'p_engine_code', 'p_origin_country', 'p_gearbox', 'p_color', 'p_state',
       'p_category', 'cf_version_name'],
      dtype='object')]

  encoding=encoding,


In [13]:
# arbitralnie ustalamy czy marka jest luxusowa lub tania

def make_cheap_luxury(df):
    df = df.copy()
    aa = df[['p_make', 'price_value']].groupby('p_make', as_index=False).agg({'price_value': ('count', 'min', 'max', 'median', 'mean', 'std')})
    aa.columns = ['p_make', 'sample_count', 'price_min', 'price_max', 'price_median', 'price_mean', 'price_std']
    lux = aa[aa['price_mean'] > 300_000]['p_make'].unique()
    chp = aa[aa['price_max'] < 100_000]['p_make'].unique()
    
    df['cf_luxury_make'] = df['p_make'].isin(lux)
    df['cf_cheap_make'] = df['p_make'].isin(chp)
    return df

tt = make_cheap_luxury(tt)

In [14]:
# arbitralnie ustalamy czy model jest luxusowy czy tani

def model_cheap_luxury(df):
    df = df.copy()
    aa = df[['p_make', 'p_model', 'price_value']].groupby(['p_make', 'p_model'], as_index=False).agg({'price_value': ('count', 'min', 'max', 'median', 'mean', 'std')})
    aa.columns = ['p_make', 'p_model', 'sample_count', 'price_min', 'price_max', 'price_median', 'price_mean', 'price_std']
    lux = aa[aa['price_mean'] > 300_000][['p_make', 'p_model']]
    
    lux['cf_luxury_model'] = True
    df = df.merge(lux, on=['p_make', 'p_model'], how='left').fillna({'cf_luxury_model': False})
    df['cf_luxury_model'] = df['cf_luxury_model'].astype(np.bool)

    chp = aa[aa['price_max'] < 50_000][['p_make', 'p_model']]
    
    chp['cf_cheap_model'] = True
    df = df.merge(chp, on=['p_make', 'p_model'], how='left').fillna({'cf_cheap_model': False})
    df['cf_cheap_model'] = df['cf_cheap_model'].astype(np.bool)
    
    return df

tt = model_cheap_luxury(tt)

In [15]:
# tt.to_hdf('car_price.dataset.v4.h5', 'data')
# tt = pd.read_hdf('car_price.dataset.v4.h5')


In [16]:
# dziwne pojemności silnika próbuję uzupełnić na podstawie 'normalnych' wartości dla marki i modelu

def adj_engine_capacity(df):
    df = df.copy()
    a = df.query('p_engine_capacity < 9999 and p_engine_capacity > 0')[['p_make', 'p_model', 'p_engine_capacity']].groupby(['p_make', 'p_model']).median().rename(columns={'p_engine_capacity': 'eng_cap'})
    return df.merge(a, on=['p_make', 'p_model'], how='left') \
        .assign(cf_engine_capacity_adj=lambda x: np.where((x['p_engine_capacity'] >= 9999) | (x['p_engine_capacity'] <= 0), x['eng_cap'], x['p_engine_capacity']).astype(np.int16)) \
        .drop(columns='eng_cap')

tt = adj_engine_capacity(tt)

In [17]:
# to samo z mocą silnika

def adj_engine_power(df):
    df = df.copy()
    a = df.query('p_engine_power < 900 and p_engine_power > 0')[['p_make', 'p_model', 'cf_engine_capacity_adj', 'p_engine_power']] \
        .groupby(['p_make', 'p_model', 'cf_engine_capacity_adj']).median().rename(columns={'p_engine_power': 'eng_pow'})
    return df.merge(a, on=['p_make', 'p_model', 'cf_engine_capacity_adj'], how='left') \
        .assign(cf_engine_power_adj=lambda x: np.where((x['p_engine_power'] >= 900) | (x['p_engine_power'] <= 0), x['eng_pow'], x['p_engine_power']).astype(np.int16)) \
        .drop(columns='eng_pow')

tt = adj_engine_power(tt)

In [18]:
# dodanie wieku i usupełnienie dziwnych przebiegów uzupełniam średnim przebiegiem (założenie 16k km rocznie)

def adj_mileage(df):
    df = df.copy()
    df['cf_age'] = np.where(df['p_year'] > 0, 2018 - df['p_year'], -1).astype(np.int8)
    df['cf_mileage_adj'] = np.where(((df['p_mileage'] >= 1500000) | (df['p_mileage'] < 0)) & (df['cf_age'] >=1), df['cf_age'] * 16000, df['p_mileage'])
    return df

tt = adj_mileage(tt)

In [19]:
# liczbę drzwi próbuję uzupełnić na podstawie typu nadwozia

def adj_doors(df):
    df = df.copy()
    df['cf_doors_adj'] = np.where((df['p_doors'] >= 2) & (df['p_doors'] < 7), df['p_doors'], -1)
    
    df.loc[(df['p_body_type'] == 'Sedan') & (df['p_doors'] == '-1'), 'cf_doors_adj'] = 4
    df.loc[(df['p_body_type'] == 'Kombi') & (df['p_doors'] == '-1'), 'cf_doors_adj'] = 5
    df.loc[(df['p_body_type'] == 'SUV') & (df['p_doors'] == '-1'), 'cf_doors_adj'] = 5
    df.loc[(df['p_body_type'] == 'Coupe') & (df['p_doors'] == '-1'), 'cf_doors_adj'] = 2
    df.loc[(df['p_body_type'] == 'Minivan') & (df['p_doors'] == '-1'), 'cf_doors_adj'] = 5
    df.loc[(df['p_body_type'] == 'Kompakt') & (df['p_doors'] == '-1'), 'cf_doors_adj'] = 5
    df.loc[(df['p_body_type'] == 'hatchback') & (df['p_doors'] == '-1'), 'cf_doors_adj'] = 5
    df.loc[(df['p_body_type'] == 'coupe/cabrio') & (df['p_doors'] == '-1'), 'cf_doors_adj'] = 2
    df.loc[(df['p_body_type'] == 'Kabriolet') & (df['p_doors'] == '-1'), 'cf_doors_adj'] = 2
    
    
    gr = ['p_make', 'p_model', 'p_version', 'p_body_type']
    fb = df[(df['p_make'] != '-1') & (df['p_model'] != '-1') & (df['p_version'] != '-1') & (df['p_body_type'] != '-1') & (df['cf_doors_adj'] != -1)][gr + ['cf_doors_adj']].groupby(gr).median().rename(columns={'cf_doors_adj':'doors'})
    df = df.merge(fb, on=gr, how='left').assign(cf_doors_adj=lambda x: np.where((x['cf_doors_adj'] == -1) & (x['doors'] > 0), x['doors'], x['cf_doors_adj'])).drop(columns='doors')

    gr = ['p_make', 'p_model', 'p_version']
    fb = df[(df['p_make'] != '-1') & (df['p_model'] != '-1') & (df['p_version'] != '-1') & (df['cf_doors_adj'] != -1)][gr + ['cf_doors_adj']].groupby(gr).median().rename(columns={'cf_doors_adj':'doors'})
    df = df.merge(fb, on=gr, how='left').assign(cf_doors_adj=lambda x: np.where((x['cf_doors_adj'] == -1) & (x['doors'] > 0), x['doors'], x['cf_doors_adj'])).drop(columns='doors')

    gr = ['p_make', 'p_model']
    fb = df[(df['p_make'] != '-1') & (df['p_model'] != '-1') & (df['cf_doors_adj'] != -1)][gr + ['cf_doors_adj']].groupby(gr).median().rename(columns={'cf_doors_adj':'doors'})
    df = df.merge(fb, on=gr, how='left').assign(cf_doors_adj=lambda x: np.where((x['cf_doors_adj'] == -1) & (x['doors'] > 0), x['doors'], x['cf_doors_adj'])).drop(columns='doors')
                           
        
    df.loc[df['car_id'] == 59057, 'cf_doors_adj'] = 3
    df['cf_doors_adj'] = df['cf_doors_adj'].astype(np.int8)
    return df

tt = adj_doors(tt)


/Users/pawelg/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [20]:
# 'faktoryzacja' cech kategorialnych

def factorize_feats(df):
    df = df.copy()
    def _fac(c, prefix='p_'):
        df[f'cfc_{c}'] = pd.factorize(df[f"{prefix}{c}"])[0]
        return df
    
    df['cf_PLN'] = df['price_currency'].map({'EUR': True, 'PLN': False})
    df = _fac('seller_type', '')
    df = _fac('body_type')
    df = _fac('color')
    df = _fac('fuel_type')
    df = _fac('gearbox')
    df = _fac('make')
    df = _fac('model')
    df = _fac('offered_by')
    df = _fac('origin_country')
    df = _fac('state')
    df = _fac('transmission')
    df = _fac('version_name', 'cf_')
    return df

tt = factorize_feats(tt)

In [21]:
# zabawa w transformacje log, pierwiastek, odwrotność

def transform_feats(df):
    df['cf_age_log'] = (np.log(df['cf_age'] + 2) - 1).astype(np.float32)
    df['cf_mileage_sqrt'] = np.where(df['cf_mileage_adj'] > 0, np.sqrt(df['cf_mileage_adj']), -1)

    df['cf_age^2'] = np.where(df['cf_age'] > 0, np.power(df['cf_age'], 2), -1).astype(np.int16)
    df['cf_age_inverted'] = np.select([df['cf_age'] > 0, df['cf_age'] == 0], [1/df['cf_age'], 2], -1).astype(np.float32)
    return df
    
tt = transform_feats(tt)

/Users/pawelg/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [22]:
tt.to_hdf('car_price.dataset.v5.h5', 'data')


/Users/pawelg/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2505: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block7_values] [items->Index(['breadcrumb', 'created_at', 'price_currency', 'seller_address',
       'seller_name', 'seller_type', 'p_model', 'p_make', 'p_transmission',
       'p_fuel_type', 'p_body_type', 'p_version', 'p_offered_by', 'p_vin',
       'p_engine_code', 'p_origin_country', 'p_gearbox', 'p_color', 'p_state',
       'p_category', 'cf_version_name'],
      dtype='object')]

  encoding=encoding,
